https://www.pingshiuanchua.com/blog/post/using-youtube-api-to-analyse-youtube-comments-on-python

## Extract YouTube Comments

### Method 1: html

In [34]:
import configparser
from urllib.parse import urlencode
import requests

import time
import json


In [2]:
config = configparser.ConfigParser()
config.read('key.properties')
key = config['google']['python.playground']

**Get Video IDs**

In [4]:
def html_get_videoid(part, maxResults, q, key=key):
    response = requests.get('https://www.googleapis.com/youtube/v3/search?' +
                            urlencode({'part': 'snippet',
                                       'maxResults':5,
                                       'q': 'Jay Chou Mojito',
                                       'key': key}))
    
    search_info = response.json()

    try:
        if response.ok :
            for i in range(maxResults):
                if search_info['items'][i]['snippet']['channelTitle'] == '周杰倫 Jay Chou':
                    video_id = search_info['items'][i]['id']['videoId']
            return video_id
    except IndexError:
        print('Failed to get status, response:', search_info)
        raise

In [5]:
video_id = html_get_videoid(part='snippet', maxResults=5, q='Jay Chou Mojito', key=key)

In [6]:
video_id

'-biOGdYiF-I'

**Get comments**

In [31]:
def html_get_comments(part, videoId, key=key):
    time.sleep(1)

    response = requests.get('https://www.googleapis.com/youtube/v3/commentThreads?' +
                        urlencode({'part':part,
                                   'videoId':videoId,
                                   'key': key}))
    resp = response.json()
    resp_tot = resp.copy()
    if 'nextPageToken' in resp:
        page_token = resp['nextPageToken']
        print("resp['nextPageToken']:", resp['nextPageToken'])
        while page_token:
            print('page_token:', page_token)
            response_tk = requests.get('https://www.googleapis.com/youtube/v3/commentThreads?' +
                        urlencode({'part':part,
                                   'pageToken':page_token,
                                   'videoId':videoId,
                                   'key': key}))
            resp_tk = response_tk.json()
            print("len(resp_tk['items']):", len(resp_tk['items']))
            for i in range(len(resp_tk['items'])):
                resp_tot['items'].append(resp_tk['items'][i])
            if 'nextPageToken' in resp_tk:
                page_token = resp_tk['nextPageToken']
                print('page_token:', page_token)
                print('===')
            else:
                break

    return resp_tot


In [32]:
comments_json = html_get_comments(part='snippet', videoId='-biOGdYiF-I', key=key)

resp['nextPageToken']: QURTSl9pMmEtbUtNdkNsdHRnVW11WGE3SEhDajZHOWNpNGpIc3ZpT0JkdFBjd2FvSmRzWk1aaVpUYkdkUzZwNVNUUG9BMU4zTEZjV2c1WC16c3l2eUVmbmxfbURxVXFhSHVDNTM2d0I2bXZvNF8yZ0NKdTJoU0tvZmlveDBPNmFucE0=
page_token: QURTSl9pMmEtbUtNdkNsdHRnVW11WGE3SEhDajZHOWNpNGpIc3ZpT0JkdFBjd2FvSmRzWk1aaVpUYkdkUzZwNVNUUG9BMU4zTEZjV2c1WC16c3l2eUVmbmxfbURxVXFhSHVDNTM2d0I2bXZvNF8yZ0NKdTJoU0tvZmlveDBPNmFucE0=
len(resp_tk['items']): 20
page_token: QURTSl9pMHhIcHYtcmd0RngtRnYzazg3VkRfRXpGaWpWNE9Gc1hlTmVBXzVmRGlXOVF6V3NqaEtJN3k1S3l0T2QtbmhWaHpXaDhKc3B6bnkyckVfWXNNdzZZMjVxUVFSR2VUNm5NVVV2bHEyZTZQeGRNRHJYMjBXWC1uZnJVQzIzWDJ0VDZnNERPTjBpTEU=
===
page_token: QURTSl9pMHhIcHYtcmd0RngtRnYzazg3VkRfRXpGaWpWNE9Gc1hlTmVBXzVmRGlXOVF6V3NqaEtJN3k1S3l0T2QtbmhWaHpXaDhKc3B6bnkyckVfWXNNdzZZMjVxUVFSR2VUNm5NVVV2bHEyZTZQeGRNRHJYMjBXWC1uZnJVQzIzWDJ0VDZnNERPTjBpTEU=
len(resp_tk['items']): 20
page_token: QURTSl9pMU93QUFkNW4xOUVEWGFrbkpUbDR1N3NSaWZCVWs0bW9PS01zYjNrSlAzUml3cFBXOHctSjdkS0hSMlZNUmpwNGVfb2I4MklmT1NSM0dTU0FvVUxYSzBGcEhXUWtyb

len(resp_tk['items']): 20
page_token: QURTSl9pMG1lV1I2RE9rSVhXd1RLam9uS2pDbEtwajNzYjRzZm5kanV2M2FqbHdLQ0ZnVFdyVXlMRVNOc1lRMXhNeFNNekdRd2lhcHhSUGhrYmR0SnlJZDRGdmhHNFFYaEQtTVZXRXd5VzBJckxxVEJCSGhKU0dvNUFLeHczRUZkdXVhU19lMHpqajNBbEZqSmc=
===
page_token: QURTSl9pMG1lV1I2RE9rSVhXd1RLam9uS2pDbEtwajNzYjRzZm5kanV2M2FqbHdLQ0ZnVFdyVXlMRVNOc1lRMXhNeFNNekdRd2lhcHhSUGhrYmR0SnlJZDRGdmhHNFFYaEQtTVZXRXd5VzBJckxxVEJCSGhKU0dvNUFLeHczRUZkdXVhU19lMHpqajNBbEZqSmc=
len(resp_tk['items']): 20
page_token: QURTSl9pMGxtWFdHQ1hwVzk4OUlaVE1pY3o2MUtHakpja2hCWjNkdHpDYnE5dnhabUZmc0NqYjJhSDg5aV84NTBldEc3bXB2R1NUTklKTzJvLWkweUJCajZiSzlnZl9KaERuTnlNVFpTNXRkVjI2RWpLc1hrSktEcTQ3a0h3M2RXSGdRc2FzaHRtNUxWZEVDN2c=
===
page_token: QURTSl9pMGxtWFdHQ1hwVzk4OUlaVE1pY3o2MUtHakpja2hCWjNkdHpDYnE5dnhabUZmc0NqYjJhSDg5aV84NTBldEc3bXB2R1NUTklKTzJvLWkweUJCajZiSzlnZl9KaERuTnlNVFpTNXRkVjI2RWpLc1hrSktEcTQ3a0h3M2RXSGdRc2FzaHRtNUxWZEVDN2c=
len(resp_tk['items']): 20
page_token: QURTSl9pMXRkaV9SZTQtVjdwMHNqMzR4VkNuZnhmN2J0b0hJaThGSFhZb2JfRC0wLT

len(resp_tk['items']): 20
page_token: QURTSl9pMXdsOG1Ka3Z0aTJtTWFCdnpSNGpfNlVsVjUwaTBrNmJnUlRUbzlOQk5BMXZPY0xYYzZ2YTRMMUwzTDBCeFJnWmtxSDJOeHhuWEV6ZVozS0RmVmZCWU1Jdzc0bFlXOGgwdGZweEFTcVdDWmI5YjRqNDA5UC1jU1dtdzRoc1d3ZThVczR5NVBaSVpMaWc=
===
page_token: QURTSl9pMXdsOG1Ka3Z0aTJtTWFCdnpSNGpfNlVsVjUwaTBrNmJnUlRUbzlOQk5BMXZPY0xYYzZ2YTRMMUwzTDBCeFJnWmtxSDJOeHhuWEV6ZVozS0RmVmZCWU1Jdzc0bFlXOGgwdGZweEFTcVdDWmI5YjRqNDA5UC1jU1dtdzRoc1d3ZThVczR5NVBaSVpMaWc=
len(resp_tk['items']): 20
page_token: QURTSl9pMWFzWlZYWVRtY3p6VDJrRWVUZ1RuUXdFMnY1Q0ZiNjdwSUJCaHhvdDd3aUNmTXJaQjJyeDIzeUhVdUh2VTVEYldBV3JueWJZaXh4SU4zY05ILThreU5Fa1Foc2s4SmJ4OUp5M1N2S2c3NW5RMXhkZDNvcGtYS2o4dTJoTnVoZk8xQmE0VTVxcTV1SEE=
===
page_token: QURTSl9pMWFzWlZYWVRtY3p6VDJrRWVUZ1RuUXdFMnY1Q0ZiNjdwSUJCaHhvdDd3aUNmTXJaQjJyeDIzeUhVdUh2VTVEYldBV3JueWJZaXh4SU4zY05ILThreU5Fa1Foc2s4SmJ4OUp5M1N2S2c3NW5RMXhkZDNvcGtYS2o4dTJoTnVoZk8xQmE0VTVxcTV1SEE=
len(resp_tk['items']): 20
page_token: QURTSl9pMVpDMTFUa3ZHanpCUkYzWkFNekJoUnRiR1lNMWRfTnp2Tjkwdmg3VG1Fdm

len(resp_tk['items']): 20
page_token: QURTSl9pMTdFY2tFbVBuMHpJSG9ldGQ5ZEVjNkhlTUk0RC1FbGxvQm9WdGhqTTRrWHRZNm9jNy1TWkpiTEFzaFMwVkJMVWM4OGc5Ulk2Vi1xejRwM3lyZ3ZNci1zdkJlZXRJWVhLX2xCa1dmR0pRTFBTU0VDNGV1cHFhMlQwLXdkVUVsZDN2a3ZhN21FQ21semc=
===
page_token: QURTSl9pMTdFY2tFbVBuMHpJSG9ldGQ5ZEVjNkhlTUk0RC1FbGxvQm9WdGhqTTRrWHRZNm9jNy1TWkpiTEFzaFMwVkJMVWM4OGc5Ulk2Vi1xejRwM3lyZ3ZNci1zdkJlZXRJWVhLX2xCa1dmR0pRTFBTU0VDNGV1cHFhMlQwLXdkVUVsZDN2a3ZhN21FQ21semc=
len(resp_tk['items']): 20
page_token: QURTSl9pMk9Wek9zSjBpVWdsYS1uTGhpejFJMUpyMUs1WWlPNzA5Zmt6aWZtNi1PWElLazQ3bGpaeXBILUxxelNLbi1Za0xaU1JETC0waTNCVFVJMXpFcXctVXVpQWc4Yk5TcHFab3hsUmtwbVk4ajVfMWg1aGlCdWhjd0RJajNNemRIalEtQlhnZkJlTkxZcnc=
===
page_token: QURTSl9pMk9Wek9zSjBpVWdsYS1uTGhpejFJMUpyMUs1WWlPNzA5Zmt6aWZtNi1PWElLazQ3bGpaeXBILUxxelNLbi1Za0xaU1JETC0waTNCVFVJMXpFcXctVXVpQWc4Yk5TcHFab3hsUmtwbVk4ajVfMWg1aGlCdWhjd0RJajNNemRIalEtQlhnZkJlTkxZcnc=
len(resp_tk['items']): 20
page_token: QURTSl9pM3h5bXUwclE1TW5zYy1yVzBpc0QyOVR3TVZMd2tCa3Z5bU1fZnR4QmhoOX

len(resp_tk['items']): 20
page_token: QURTSl9pMjdBSGVMTko4azRDcnIxS1FYdW5ZUTRVMkdXVlUwWW85VUdIRjVJbTRUdTkwYTI1U1JyV1pKYXFjSFQtUjNCSVNsd2VXZ0lESXlQSm5CUDVXc19xbEhuUThyX3VMNlkyQUZFZmFwdGhqaXZoRGU3QzVwQkF4Ni0wdFhOSEtKRmluOXNrOHpmSDZwZkE=
===
page_token: QURTSl9pMjdBSGVMTko4azRDcnIxS1FYdW5ZUTRVMkdXVlUwWW85VUdIRjVJbTRUdTkwYTI1U1JyV1pKYXFjSFQtUjNCSVNsd2VXZ0lESXlQSm5CUDVXc19xbEhuUThyX3VMNlkyQUZFZmFwdGhqaXZoRGU3QzVwQkF4Ni0wdFhOSEtKRmluOXNrOHpmSDZwZkE=
len(resp_tk['items']): 20
page_token: QURTSl9pMkNVRGx1aWFuUmZWMUlHSUU5X2FUcUZseDd2cDBHbUlVaUtfMVpPLVNONmVWaDJ4TUxCenR5Y0RYdHNQaU9RNTk3WEMzZmtsRzNQVDlWYWw5Mk9EbHFaR1JlQzkxSVRLRVZYQXR5R0EwVjl3N2ZjTENHSUhmbEJURXJHR1RwS1R2YkhEZmx0UXE3T2c=
===
page_token: QURTSl9pMkNVRGx1aWFuUmZWMUlHSUU5X2FUcUZseDd2cDBHbUlVaUtfMVpPLVNONmVWaDJ4TUxCenR5Y0RYdHNQaU9RNTk3WEMzZmtsRzNQVDlWYWw5Mk9EbHFaR1JlQzkxSVRLRVZYQXR5R0EwVjl3N2ZjTENHSUhmbEJURXJHR1RwS1R2YkhEZmx0UXE3T2c=
len(resp_tk['items']): 20
page_token: QURTSl9pMG0xZlVBWVVpSUI1ZGVjOHFYVm9tcVlISnJQNzZiT1JOR2F2VnRUbElyTU

...

len(resp_tk['items']): 20
page_token: QURTSl9pM2VZUk5iQk5zd1VQbS04ZzhWODlUOVNsMjc4aWhnTlpFOHFjX3ZFd185bGw1RjJ0alpkbkNnRGxUZ3g2WHEyX3A2TUxVS0ZyejhFMm51Z3pDS1NySXNCVVpmenJyNUZRV1FoQ2FmdnEzM3ZJVGU5VURWYlk0ZDV3UUNQMjBIRzFDV3dpTlVUVkFLREFj
===
page_token: QURTSl9pM2VZUk5iQk5zd1VQbS04ZzhWODlUOVNsMjc4aWhnTlpFOHFjX3ZFd185bGw1RjJ0alpkbkNnRGxUZ3g2WHEyX3A2TUxVS0ZyejhFMm51Z3pDS1NySXNCVVpmenJyNUZRV1FoQ2FmdnEzM3ZJVGU5VURWYlk0ZDV3UUNQMjBIRzFDV3dpTlVUVkFLREFj
len(resp_tk['items']): 20
page_token: QURTSl9pMUdnSW5jd2xDZzJQRWxPV25BakRSSUMzVld6d0c3WUFGMmJ5eTFLRTZKNGtiVzI3VE1VY1hfYXdtNEd2NklLeXNLVGp3Qms4VHExNmV0X2NXTklONXRUb05CZWM0NVItTllBZWlKZkhPRE9fbHBySnVzd2FXeUFHZnRCWERaS1Q3NWR2NVBYZ2JMaVdj
===
page_token: QURTSl9pMUdnSW5jd2xDZzJQRWxPV25BakRSSUMzVld6d0c3WUFGMmJ5eTFLRTZKNGtiVzI3VE1VY1hfYXdtNEd2NklLeXNLVGp3Qms4VHExNmV0X2NXTklONXRUb05CZWM0NVItTllBZWlKZkhPRE9fbHBySnVzd2FXeUFHZnRCWERaS1Q3NWR2NVBYZ2JMaVdj
len(resp_tk['items']): 20
page_token: QURTSl9pMHc5dFpUcGZGdkg2TC1QMm03eEFZS1lsbFA1NDFZZ2FCSG1QSHU4cmVPV1

len(resp_tk['items']): 20
page_token: QURTSl9pMVhQdlY1dTRXN3ZzVU90RHJJcWl6dEJMZXRBcGtCV0UxRlhzbHJoRXliR3g1amY2ektzU3g3b1hlS09kLW92MXpnTFZ0M05xRVVkZmFlbDllN1Y0Qk12QlFxa0tMWVZQTk42Y2xXWk5XbzhiUzhDakNwMVk2dHBxaFRmQURwR0FISWNGMGlYOUx0U2Mw
===
page_token: QURTSl9pMVhQdlY1dTRXN3ZzVU90RHJJcWl6dEJMZXRBcGtCV0UxRlhzbHJoRXliR3g1amY2ektzU3g3b1hlS09kLW92MXpnTFZ0M05xRVVkZmFlbDllN1Y0Qk12QlFxa0tMWVZQTk42Y2xXWk5XbzhiUzhDakNwMVk2dHBxaFRmQURwR0FISWNGMGlYOUx0U2Mw
len(resp_tk['items']): 20
page_token: QURTSl9pMGEwRFFYdGtFRzFUVi1Ob1FVMDFYTUtXUXRObnRqZG5yS2ZoYmx0cWlidzlxQ2hvQmJablRIZDBOVkp4X0UtUlBjMXl3dl9VMWZfbTFpbk01Yld3YWl2RzRTUnFGekFCTW1zVnFuVWx3U2JvXzFjRFJDc0ZCLW9lM1BjN2hDZjJqRVYzZktqNmJoNEhz
===
page_token: QURTSl9pMGEwRFFYdGtFRzFUVi1Ob1FVMDFYTUtXUXRObnRqZG5yS2ZoYmx0cWlidzlxQ2hvQmJablRIZDBOVkp4X0UtUlBjMXl3dl9VMWZfbTFpbk01Yld3YWl2RzRTUnFGekFCTW1zVnFuVWx3U2JvXzFjRFJDc0ZCLW9lM1BjN2hDZjJqRVYzZktqNmJoNEhz
len(resp_tk['items']): 20
page_token: QURTSl9pMmk0YllVT2NBb3E3WGdBb3JETGFGT005NWxqS0dwcmRrMGFqUHZxUXdSd2

len(resp_tk['items']): 20
page_token: QURTSl9pMXdPLUVRNlZyaUdGZ2tQaHdua3JhY2dtQk80Wm9sZ2p0aTE0Um90eTZvRWw4SVZEVF9mSldjcl83b0xPS0NmWUNyWHdvX1ZBRmJsaWhIRjR5aThMSGNyTlhvOXhVb2ZZRGdCVDN4S1FVRlEtaU1CYUNVbktBazRwN2pBYjJkbGt0MFZSZGk0cWJ0RU9v
===
page_token: QURTSl9pMXdPLUVRNlZyaUdGZ2tQaHdua3JhY2dtQk80Wm9sZ2p0aTE0Um90eTZvRWw4SVZEVF9mSldjcl83b0xPS0NmWUNyWHdvX1ZBRmJsaWhIRjR5aThMSGNyTlhvOXhVb2ZZRGdCVDN4S1FVRlEtaU1CYUNVbktBazRwN2pBYjJkbGt0MFZSZGk0cWJ0RU9v
len(resp_tk['items']): 20
page_token: QURTSl9pM3VIZ2pZb3YySk9GWHdDbjBjR0JoRkI5czBHWDl5T0VEeWJwcGcySVpwQXJudHFwa1JNVjZjZHpqNGNpeXZJclRlclFRZzJwUUVmQzFNOHpTaDZabW1hZFdrZndFZEtIR0JESlByX1UzRTQ0ZGlNWkg5SWVVSUZmRDBBR3BKajRuaUNKeVhpa0hhRkw0
===
page_token: QURTSl9pM3VIZ2pZb3YySk9GWHdDbjBjR0JoRkI5czBHWDl5T0VEeWJwcGcySVpwQXJudHFwa1JNVjZjZHpqNGNpeXZJclRlclFRZzJwUUVmQzFNOHpTaDZabW1hZFdrZndFZEtIR0JESlByX1UzRTQ0ZGlNWkg5SWVVSUZmRDBBR3BKajRuaUNKeVhpa0hhRkw0
len(resp_tk['items']): 20
page_token: QURTSl9pMGpEZ0lFQWVQS1JlOElCV3VYWktiY0tzblhFZG1hZWMwN19uZGszQmJUZz

len(resp_tk['items']): 20
page_token: QURTSl9pM2ZHYmxwRXY3UlZhNngweGxhMmFqOFItOWpfNmpweDh2TmJ4SHlhNzM2bm51c01NbUNvTlVqYXpad0lsdTJ2U21za09aX2xVczJtaEJLRm5ySS04RGQ4V0xTVUFHOTRpRy1LSG5yNnFrSy1hNlI3a2gydHcxekNzNllhVnhucVAzYkt3M0pUUVgzX0xr
===
page_token: QURTSl9pM2ZHYmxwRXY3UlZhNngweGxhMmFqOFItOWpfNmpweDh2TmJ4SHlhNzM2bm51c01NbUNvTlVqYXpad0lsdTJ2U21za09aX2xVczJtaEJLRm5ySS04RGQ4V0xTVUFHOTRpRy1LSG5yNnFrSy1hNlI3a2gydHcxekNzNllhVnhucVAzYkt3M0pUUVgzX0xr
len(resp_tk['items']): 20
page_token: QURTSl9pMWFpRDQzeEczUWk1dW1QY0pMb09vTzZ0WTZPMTFJRHVEX3JNTHl6bGxuSmFBbm9lOGxTamlwNjhsUHBwT085QWxkS3o4bHYzUDhpYURlcHp3TW5DekVMNURXN1RYUTczcWxRX0c0X2lpMGNObndVYlpxcEd0N01ZeHRFRjNpTGtOaWhRMXlpTUNtNGNz
===
page_token: QURTSl9pMWFpRDQzeEczUWk1dW1QY0pMb09vTzZ0WTZPMTFJRHVEX3JNTHl6bGxuSmFBbm9lOGxTamlwNjhsUHBwT085QWxkS3o4bHYzUDhpYURlcHp3TW5DekVMNURXN1RYUTczcWxRX0c0X2lpMGNObndVYlpxcEd0N01ZeHRFRjNpTGtOaWhRMXlpTUNtNGNz
len(resp_tk['items']): 20
page_token: QURTSl9pMkVWTEFCbXFRTDNIYTdDSUZWV2w1OG53WmtXM00zTUdsZmZwU0dDYlRHb2

len(resp_tk['items']): 20
page_token: QURTSl9pM2owa0p1SlFUckNPTVlweHZUZjlPSXVaZkUzZ2xQd1lRTS1rWEpYTU9MSWpPdnFJSThpMDBYUGtQY3F4dEZVMmJCVUdKMWJRdnZ5UUg0dTd4VjVVNXRMQjhEamcwckxFMDNNVUlVRHltREJRVkJza3E2amtCT1A2aUlqRzItVmU5T2E2cDl6WEE1QnFZ
===
page_token: QURTSl9pM2owa0p1SlFUckNPTVlweHZUZjlPSXVaZkUzZ2xQd1lRTS1rWEpYTU9MSWpPdnFJSThpMDBYUGtQY3F4dEZVMmJCVUdKMWJRdnZ5UUg0dTd4VjVVNXRMQjhEamcwckxFMDNNVUlVRHltREJRVkJza3E2amtCT1A2aUlqRzItVmU5T2E2cDl6WEE1QnFZ
len(resp_tk['items']): 20
page_token: QURTSl9pMlVtd3BDT2VmajlSdW1lWF8zTmNrTDdib25BX0RqVjlWRU8yTGNzbmw0dFhpQkRGd3dHdlRjc2FpSnJHUGJ0NE5lUTdpU2VtMFNWOTFZeTJMaTRGRUJfNF9ITXFNNTFZbmRhdUpGVTE0czItd0U3aTRhcmlpTk5HV0J2VDRSRERURzUwQXJLbDhSQXhn
===
page_token: QURTSl9pMlVtd3BDT2VmajlSdW1lWF8zTmNrTDdib25BX0RqVjlWRU8yTGNzbmw0dFhpQkRGd3dHdlRjc2FpSnJHUGJ0NE5lUTdpU2VtMFNWOTFZeTJMaTRGRUJfNF9ITXFNNTFZbmRhdUpGVTE0czItd0U3aTRhcmlpTk5HV0J2VDRSRERURzUwQXJLbDhSQXhn
len(resp_tk['items']): 20
page_token: QURTSl9pMU9qOFJnUE1rb1k4YmpFaVFHanNJcTFkdEk0V2QwZHp6RGlkWktaN2pXY1

In [38]:
len(comments_json['items'])

10194

In [35]:
with open('comments_json.json', 'w') as json_file:
    json.dump(comments_json, json_file)

### Method 2: python

**Get Video IDs**

In [7]:
import os
import googleapiclient.discovery

In [8]:
def py_get_videoid(part, maxResults, q, key=key):
    api_service_name = 'youtube'
    api_version = 'v3'

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = key)

    request = youtube.search().list(
        part=part,
        maxResults=maxResults,
        q=q
    )
    search_info = request.execute()

    try:
        for i in range(maxResults):
            if search_info['items'][i]['snippet']['channelTitle'] == '周杰倫 Jay Chou':
                video_id = search_info['items'][i]['id']['videoId']
        return video_id
    except IndexError:
        print('Failed to get status, response:', search_info)
        raise

In [9]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

video_id = py_get_videoid(part='snippet', maxResults=5, q='Jay Chou Mojito', key=key)

In [10]:
video_id

'-biOGdYiF-I'

**Get comments**

In [43]:
def py_get_comments(part, videoId, key=key):
    time.sleep(1)

    api_service_name = 'youtube'
    api_version = 'v3'

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = key)
    
    request = youtube.commentThreads().list(
        part=part,
        videoId=videoId
    )
    resp = request.execute()
    resp_tot = resp.copy()

    if 'nextPageToken' in resp:
        page_token = resp['nextPageToken']
        print("resp['nextPageToken']:", resp['nextPageToken'])
        while page_token:
            print('page_token:', page_token)
            response_tk = youtube.commentThreads().list(
                part=part,
                pageToken=page_token,
                videoId=videoId
            )
            resp_tk = response_tk.execute()
            print("len(resp_tk['items']):", len(resp_tk['items']))
            for i in range(len(resp_tk['items'])):
                resp_tot['items'].append(resp_tk['items'][i])
            if 'nextPageToken' in resp_tk:
                page_token = resp_tk['nextPageToken']
                print('page_token:', page_token)
                print('===')
            else:
                break

    return resp_tot


In [44]:
comments_json = html_get_comments(part='snippet', videoId='-biOGdYiF-I', key=key)

resp['nextPageToken']: QURTSl9pM1d2b3pmQWVrbDhEQ0J4dnZqVlk4OFl0ekpEbF9Jc1E2aG96cExySTc4bmloVU1HeG1qZGxKY0l5YWVhamc2djhWSUlFb2RGYkFyMFRoWnRHTGR2M2Z0NlFZcTZJanVsU0RRazc3UHlHYnNsZHYzLTJic2dfRmtXX2M=
page_token: QURTSl9pM1d2b3pmQWVrbDhEQ0J4dnZqVlk4OFl0ekpEbF9Jc1E2aG96cExySTc4bmloVU1HeG1qZGxKY0l5YWVhamc2djhWSUlFb2RGYkFyMFRoWnRHTGR2M2Z0NlFZcTZJanVsU0RRazc3UHlHYnNsZHYzLTJic2dfRmtXX2M=
len(resp_tk['items']): 20
page_token: QURTSl9pMjlYM19LTVdhekh5eXp1eDVmZTdDVzdhRG1KdXpEWS0zZF9NNmtFX1BKSURDRUU5X0pkejBQZzhWd01xWGZQYnplTFJ2OVNjRVpPYk5wNnAzNDlfcFRpSFhCTENnWndaaGJVTkppOEFwUzlpLUN6c0tDRnFCS3haeGVHWmlhZVRXbTNrTkE=
===
page_token: QURTSl9pMjlYM19LTVdhekh5eXp1eDVmZTdDVzdhRG1KdXpEWS0zZF9NNmtFX1BKSURDRUU5X0pkejBQZzhWd01xWGZQYnplTFJ2OVNjRVpPYk5wNnAzNDlfcFRpSFhCTENnWndaaGJVTkppOEFwUzlpLUN6c0tDRnFCS3haeGVHWmlhZVRXbTNrTkE=
len(resp_tk['items']): 20
page_token: QURTSl9pM2JRX1E5RnU2MDdfams5RWFXQWw5eVlJQ1Q5dnJfV1NnVDBNQmlCTEx2N0tPS3gzY2ZUQ2NWQmdPczRiZmRoUHY1SGEybWFOeDE5eW1MbjlQV0N1ajRITDdyQTRMOEY4cGRiQVFOelNrM

len(resp_tk['items']): 20
page_token: QURTSl9pMmdFTjdPOUw2QVJoWWhUNVgtdENBdGU0VlVOQXYwMU42djdkUldIbGVmM1RJdzdDSzlaUDFDSmlBWXVlSHBMeFd0dnNjYm1WRFlRQUNUeTZYLUJoaDI5SHE4QldmSmZ3ZWxKbl96bTZyTGo5SzRuMUluYVFEbUgtNG5HczJ6emJnU2hHMk5jNDQ=
===
page_token: QURTSl9pMmdFTjdPOUw2QVJoWWhUNVgtdENBdGU0VlVOQXYwMU42djdkUldIbGVmM1RJdzdDSzlaUDFDSmlBWXVlSHBMeFd0dnNjYm1WRFlRQUNUeTZYLUJoaDI5SHE4QldmSmZ3ZWxKbl96bTZyTGo5SzRuMUluYVFEbUgtNG5HczJ6emJnU2hHMk5jNDQ=
len(resp_tk['items']): 20
page_token: QURTSl9pMmd3a01yQmdjT1ZVZ3RucWtzMk5pMEQ4ZzFqOW9rcGk5YTlzUHhQamxVeFpvRDM3cDNwLURicjZiaU14cHZnTkxrX3JXTTMycE9uTUtub2VUMjVqOFhWdE5uU3RKaGhub2F4VUcteEVXbDBoOXFRcEJaNXFMekVORjN1Y19VTE11Q3JNTE1IeGs=
===
page_token: QURTSl9pMmd3a01yQmdjT1ZVZ3RucWtzMk5pMEQ4ZzFqOW9rcGk5YTlzUHhQamxVeFpvRDM3cDNwLURicjZiaU14cHZnTkxrX3JXTTMycE9uTUtub2VUMjVqOFhWdE5uU3RKaGhub2F4VUcteEVXbDBoOXFRcEJaNXFMekVORjN1Y19VTE11Q3JNTE1IeGs=
len(resp_tk['items']): 20
page_token: QURTSl9pM19jMDJlQ2s1aTljLWVxTnlYLTh1dTNMYWE1OVJ0eFFEdzRPRldUdHdfcUlmTjZuVlpjUVc0RF

len(resp_tk['items']): 20
page_token: QURTSl9pMHpMTFBjanV5eTl6UVhHNF9CdHBIaXJNOG9XUk83Rk9BcWtOUm9iaTBHWGxRZUdaRWNJbklvUW9zMU0tcDdTT215MW1aTXFJeWlUdzZ3WW5ZV1hrMDZteVd5MlJVdWNIeHhkMEZwZmZOeVlvTHhOUWJtUS0yckVBbWtDZTJ3YlNOTUc5cUU3bkE=
===
page_token: QURTSl9pMHpMTFBjanV5eTl6UVhHNF9CdHBIaXJNOG9XUk83Rk9BcWtOUm9iaTBHWGxRZUdaRWNJbklvUW9zMU0tcDdTT215MW1aTXFJeWlUdzZ3WW5ZV1hrMDZteVd5MlJVdWNIeHhkMEZwZmZOeVlvTHhOUWJtUS0yckVBbWtDZTJ3YlNOTUc5cUU3bkE=
len(resp_tk['items']): 20
page_token: QURTSl9pM25TNkFUa2dyakNueEJoLWc2MkY5Ujl1Y25sRWI5ckhqbk9mRlp1cDUxNGhOTExnVkVFTzFDaUhSU2g2MTNIR1NpUXFiSFNOeW9fZXUxQmpKRXpjMUhxOTlkc3AzdVd2NG1WVXV2cUZLbmJSeFI0NXR4elVYd05XaC1KOUNmbWFvay0wVXJmZlU=
===
page_token: QURTSl9pM25TNkFUa2dyakNueEJoLWc2MkY5Ujl1Y25sRWI5ckhqbk9mRlp1cDUxNGhOTExnVkVFTzFDaUhSU2g2MTNIR1NpUXFiSFNOeW9fZXUxQmpKRXpjMUhxOTlkc3AzdVd2NG1WVXV2cUZLbmJSeFI0NXR4elVYd05XaC1KOUNmbWFvay0wVXJmZlU=
len(resp_tk['items']): 20
page_token: QURTSl9pMUM3OVZTdFZVNTlYdUZZX1F2ZnFIWWVsNTlHM2dCNEI3UVNyR3hJeHZYYTExNnY3TVNjTVlpRn

len(resp_tk['items']): 20
page_token: QURTSl9pMnI5OTJQU19LaXd1Y2l3WWo5d0x3U0tPS3YtNzZON09HS28tVHUwUFZhN3FiNlVtMlgyWlR4aUoyTW1BejVfRzhzVUtFWTdCcUxPZGJhMmhhQkdfWVY4WnA2ZGJLT3hVRkdtekRhRFJuTGxIcWJERGlqbDQ4T2FOWHd6MW1YYjM5cmFnYXd6VGs=
===
page_token: QURTSl9pMnI5OTJQU19LaXd1Y2l3WWo5d0x3U0tPS3YtNzZON09HS28tVHUwUFZhN3FiNlVtMlgyWlR4aUoyTW1BejVfRzhzVUtFWTdCcUxPZGJhMmhhQkdfWVY4WnA2ZGJLT3hVRkdtekRhRFJuTGxIcWJERGlqbDQ4T2FOWHd6MW1YYjM5cmFnYXd6VGs=
len(resp_tk['items']): 20
page_token: QURTSl9pM3d0WnVxVE1fbGlXRUJ5cm9ZVGxNQ2toWnRfX0ZYS083WUdkSUdoLWFsa2JMSnQ3UlAwQVZHbHd4dGUtXzlnYjZORkJDcGdZcWNYQjRLZDFxajRYSWxvWll5b1ZqM0sybmtlSkhTbktwd3A3dl8wa2hQaF9xaXo0N1lmZVg0OUdTNzd2NmFPelk=
===
page_token: QURTSl9pM3d0WnVxVE1fbGlXRUJ5cm9ZVGxNQ2toWnRfX0ZYS083WUdkSUdoLWFsa2JMSnQ3UlAwQVZHbHd4dGUtXzlnYjZORkJDcGdZcWNYQjRLZDFxajRYSWxvWll5b1ZqM0sybmtlSkhTbktwd3A3dl8wa2hQaF9xaXo0N1lmZVg0OUdTNzd2NmFPelk=
len(resp_tk['items']): 20
page_token: QURTSl9pMklZZDVhLTVqZTVkTFpKWkZMdEJ5dGxRVFh1ckR4dlJjWkxXbjk1Ukl1eUpQM2hzYTlWOXNGQ1

len(resp_tk['items']): 20
page_token: QURTSl9pMkstZE04eTBRSmtXcGZtWjM5NzR5VVVLeUptZms2Y3prY2loT1hza2pQRlB4V1dYRlppclkxcWFyTTRIclhObWlJMF9STVZXRDVWRGhLdi1aX0kyMXZVcW1QTHNOMVZ2dTFiZ3R4LWRzN0t5bEM2YVBYLVlJWklhbFRQcEw1dlFtWFYxLUJIUnc=
===
page_token: QURTSl9pMkstZE04eTBRSmtXcGZtWjM5NzR5VVVLeUptZms2Y3prY2loT1hza2pQRlB4V1dYRlppclkxcWFyTTRIclhObWlJMF9STVZXRDVWRGhLdi1aX0kyMXZVcW1QTHNOMVZ2dTFiZ3R4LWRzN0t5bEM2YVBYLVlJWklhbFRQcEw1dlFtWFYxLUJIUnc=
len(resp_tk['items']): 20
page_token: QURTSl9pMm9GUkR6ajFNMmxlVjlJeEo5V25tT3pkaGhNMGVKdEdCVDVhandvT2dmVEczdThfb0RWcTZNQ1ltbVZmTkcwZ0Y1WFJoV201NDRrZzlxWDlNQkMwYTU3NDJFaXRGeHFjdXdoY0xGMzBjd3FxUEVPZmNmTnFhR2w4OXNEYjB2dHo2clNENWpiTWM=
===
page_token: QURTSl9pMm9GUkR6ajFNMmxlVjlJeEo5V25tT3pkaGhNMGVKdEdCVDVhandvT2dmVEczdThfb0RWcTZNQ1ltbVZmTkcwZ0Y1WFJoV201NDRrZzlxWDlNQkMwYTU3NDJFaXRGeHFjdXdoY0xGMzBjd3FxUEVPZmNmTnFhR2w4OXNEYjB2dHo2clNENWpiTWM=
len(resp_tk['items']): 20
page_token: QURTSl9pMkRPZmFrQm5zNlltaUVTaHVIbVhYMVNrRC1RcldlZkRxVloySGpnd2VCLVpqUDdxcV90Z1JET3

...

len(resp_tk['items']): 20
page_token: QURTSl9pM0hRenViQmJwa3lBSl9Qd0VQZl9ROHBpSXJWUEFWQ0hPRWxXQm9lUDU5a3dZekhJWmNyVHpzc2k5b2M5YzB0VjRwY1A3dzFzaUZ6U21GYUhLcW5vTFkzSnEwMFR6NEhuLVlOYy1NX2pXenpDSHJhc2cyZHg3Rk5USjZmeUhCTmdrZXNoaC02OFd0
===
page_token: QURTSl9pM0hRenViQmJwa3lBSl9Qd0VQZl9ROHBpSXJWUEFWQ0hPRWxXQm9lUDU5a3dZekhJWmNyVHpzc2k5b2M5YzB0VjRwY1A3dzFzaUZ6U21GYUhLcW5vTFkzSnEwMFR6NEhuLVlOYy1NX2pXenpDSHJhc2cyZHg3Rk5USjZmeUhCTmdrZXNoaC02OFd0
len(resp_tk['items']): 20
page_token: QURTSl9pM0k3dXlFRVNVNGZVZi1ha0dteWV6NnFmdXpHWVM2djhhVE5nMDQ0UFdicnZ6VzNXeHBhYm9DT1VfWUM3anBNX2syNTFTVG5wNjdRbFN0NjFqOEM4U284YlBXR25tMW51NnFCX05iT0FWR2xmdWNEeURKaE9qbWduWHltOVdHUXlXejBBNHlhM1Vv
===
page_token: QURTSl9pM0k3dXlFRVNVNGZVZi1ha0dteWV6NnFmdXpHWVM2djhhVE5nMDQ0UFdicnZ6VzNXeHBhYm9DT1VfWUM3anBNX2syNTFTVG5wNjdRbFN0NjFqOEM4U284YlBXR25tMW51NnFCX05iT0FWR2xmdWNEeURKaE9qbWduWHltOVdHUXlXejBBNHlhM1Vv
len(resp_tk['items']): 20
page_token: QURTSl9pMGw3WW1IMlJFVXY0LXdQRjh3dmhsUWpHZU5JX2NqVTB3enVyTkdtN1pPNTItRTM0eFR0emxQZE

len(resp_tk['items']): 20
page_token: QURTSl9pMU9ZQnNKQ3ZncmFYbk5va2Y0VTdydGRLVDF5a3h5T3dualNMV3JYbGVWckw3NWdkN1BUT1hVN1FFQ3VPRDVhRFdaN292TF9YTEpBVUU1NzVLZHZucEVnSFhKNmFpR3hMSHBrREFreFhhVEVfQ2ZhWjRjRnE2MkVoVl9lV3NEM18weF85dE1YdWRU
===
page_token: QURTSl9pMU9ZQnNKQ3ZncmFYbk5va2Y0VTdydGRLVDF5a3h5T3dualNMV3JYbGVWckw3NWdkN1BUT1hVN1FFQ3VPRDVhRFdaN292TF9YTEpBVUU1NzVLZHZucEVnSFhKNmFpR3hMSHBrREFreFhhVEVfQ2ZhWjRjRnE2MkVoVl9lV3NEM18weF85dE1YdWRU
len(resp_tk['items']): 20
page_token: QURTSl9pMTVieTZyTzlBNHJSSFMwQmFUbHhnOXYxajBXNkc1YmJYZEtwNjh2UTJxZlNIbzh4VDV5S2hhUlUxd1otNS1KeWVWLTZ5R1YtYnRmekYwTGpReF8xU0E1bl9SRnVlT2ZSS1psSlFkQ1c3NDFWUnBXRXQ0d0dnRG44VHdRdDZvWWhaMzczdFM0SzZh
===
page_token: QURTSl9pMTVieTZyTzlBNHJSSFMwQmFUbHhnOXYxajBXNkc1YmJYZEtwNjh2UTJxZlNIbzh4VDV5S2hhUlUxd1otNS1KeWVWLTZ5R1YtYnRmekYwTGpReF8xU0E1bl9SRnVlT2ZSS1psSlFkQ1c3NDFWUnBXRXQ0d0dnRG44VHdRdDZvWWhaMzczdFM0SzZh
len(resp_tk['items']): 20
page_token: QURTSl9pM3p0aTZHNVVnVzRRVXdjUU5OWVpiSVBUWGVleDFiYUMzOHZDNUVvMzlKOERRWVZSaEV0eUxQcE

len(resp_tk['items']): 20
page_token: QURTSl9pMUN5S3RMRFZBc2dlcENuZTF2bEpLR2s0UFpWUFgwOGc5dUNST28zTUtSeFgyRFB2TG1xTFdiam0zUVhERW14UmRBTEJPSjVrUE5RUTE5eE5kRDlJUDdXNnVpMC1SOEJmU1JqOHFmNkVtSmxEa1VZc1pWUVVmU2xZZFFWNHVDLWdKamQzcjFieDl5
===
page_token: QURTSl9pMUN5S3RMRFZBc2dlcENuZTF2bEpLR2s0UFpWUFgwOGc5dUNST28zTUtSeFgyRFB2TG1xTFdiam0zUVhERW14UmRBTEJPSjVrUE5RUTE5eE5kRDlJUDdXNnVpMC1SOEJmU1JqOHFmNkVtSmxEa1VZc1pWUVVmU2xZZFFWNHVDLWdKamQzcjFieDl5
len(resp_tk['items']): 20
page_token: QURTSl9pME9HSEJ5dHZHbEFjajl4dTJhdlV6Um9BTnByZG9Ka3ZGakhSbXhmbVM0WmU2Y3dkX1A3Wm83LVQ0WXVLSFdEOGRDN1F0eTh1MllOOHNfeEtRUTRqUmRab2RBbHZQWjR0YjlLdEkwWHYxbGhvUG5KeThjZEsyelR4a2RKYklJT0pOaFBrZk5lMU9y
===
page_token: QURTSl9pME9HSEJ5dHZHbEFjajl4dTJhdlV6Um9BTnByZG9Ka3ZGakhSbXhmbVM0WmU2Y3dkX1A3Wm83LVQ0WXVLSFdEOGRDN1F0eTh1MllOOHNfeEtRUTRqUmRab2RBbHZQWjR0YjlLdEkwWHYxbGhvUG5KeThjZEsyelR4a2RKYklJT0pOaFBrZk5lMU9y
len(resp_tk['items']): 20
page_token: QURTSl9pMmVCcVhPalVKYjJha3VDZnpBRDNDM0xVemFQdU0yVjVWRlZOVzFCUlB2RVduMXRSdEp2SWxQcH

len(resp_tk['items']): 20
page_token: QURTSl9pM1hMYjJLUm81RGFEODJyY0xJLU1scDRIdUJMTE1kUGhDNUR1WDM3QnEtdUg2d19VYXRiMmdXY0N2MzFrUVZYZmhmVHZNQnN5cmZ4OXBmM0trNUQ3cGFoWHFlLXN0Y1Jlbk5OQVFxX2xfb1pWejViZzBpMW50NUNSQTlGQV9OSzk1N1NDbE9VRVpD
===
page_token: QURTSl9pM1hMYjJLUm81RGFEODJyY0xJLU1scDRIdUJMTE1kUGhDNUR1WDM3QnEtdUg2d19VYXRiMmdXY0N2MzFrUVZYZmhmVHZNQnN5cmZ4OXBmM0trNUQ3cGFoWHFlLXN0Y1Jlbk5OQVFxX2xfb1pWejViZzBpMW50NUNSQTlGQV9OSzk1N1NDbE9VRVpD
len(resp_tk['items']): 20
page_token: QURTSl9pMUpnZ2tzSEc5WHFHc0RMRTVNelRpQXdkUUJtbDBoeWxfQ0tCb1lxQTNjZmlUR1dPTldnUGpSbWJmQXRYTGhyREVHcnBtTzdsb2l5T3JzNVlpMWtwdTJRUmFZUzllM2w2dVlnYWNVR2VuQU9nd3hET0VMZHduaHdpWVNXQkZSYlgyTkFYNWMyRUI5
===
page_token: QURTSl9pMUpnZ2tzSEc5WHFHc0RMRTVNelRpQXdkUUJtbDBoeWxfQ0tCb1lxQTNjZmlUR1dPTldnUGpSbWJmQXRYTGhyREVHcnBtTzdsb2l5T3JzNVlpMWtwdTJRUmFZUzllM2w2dVlnYWNVR2VuQU9nd3hET0VMZHduaHdpWVNXQkZSYlgyTkFYNWMyRUI5
len(resp_tk['items']): 20
page_token: QURTSl9pM3o1U2Y0eFpjQTZETF94cFo3bG5lZU1YbDhsQUtUQmg5UVlxTy1ka3dMVFVVbmRrTk9CaHY1ck

len(resp_tk['items']): 20
page_token: QURTSl9pMngwLUQ0V2FlTzlMYTFjX1k4elE4VHVMSVVROWNOQUF0cndsdWdzcVY3ektRbGtRd21nMjRKR0NFQWlKa2ZXQWJlNUF1RlA2dnpoLTZoUU52QmVJMFdhVkFQZkdGdG1FYXJsZWw5aWgyMkRzTGdKUXRXS0FVXzI3SVh4ZDZqWTZROVVMa2FaeDQ2
===
page_token: QURTSl9pMngwLUQ0V2FlTzlMYTFjX1k4elE4VHVMSVVROWNOQUF0cndsdWdzcVY3ektRbGtRd21nMjRKR0NFQWlKa2ZXQWJlNUF1RlA2dnpoLTZoUU52QmVJMFdhVkFQZkdGdG1FYXJsZWw5aWgyMkRzTGdKUXRXS0FVXzI3SVh4ZDZqWTZROVVMa2FaeDQ2
len(resp_tk['items']): 20
page_token: QURTSl9pMWVNUHVYeXp1ZEVodmYwb2J0LUJPelV2VjQ4bEVwdnAxU2lOcC0yQmtWR1UtSDB5R19kZ1lwVm9xeHJFWkJTZ1JwTVJqaFZoYmRFMHplLXRrdndBVjZKNTBwbENSdE5xVHlUeG9GbGg1QmRjbzZiTTdQeUNsZDB2TUZGa2tGc3RYVTVPSkJiY25Z
===
page_token: QURTSl9pMWVNUHVYeXp1ZEVodmYwb2J0LUJPelV2VjQ4bEVwdnAxU2lOcC0yQmtWR1UtSDB5R19kZ1lwVm9xeHJFWkJTZ1JwTVJqaFZoYmRFMHplLXRrdndBVjZKNTBwbENSdE5xVHlUeG9GbGg1QmRjbzZiTTdQeUNsZDB2TUZGa2tGc3RYVTVPSkJiY25Z
len(resp_tk['items']): 20
page_token: QURTSl9pMTdDMG1VZjJ1OE5CWlFWVGkydnpDcXBqVUl6YVRPWUxFczhGR0Q0bU8yTEwyOEY0cDctZXY3ak

In [46]:
len(comments_json['items'])

10197

In [47]:
with open('comments_json.json', 'w') as json_file:
    json.dump(comments_json, json_file)